# TP7 : Programmation par contrat, programmation défensive et exceptions

## Programmation par contrat

Lors du précédent cours théorique, nous avons vu la notion de programmation par contrat.  

Nous vous invitons à relire la synthèse sur le sujet, disponible sur Moodle, avant de passer à la réalisation de ce TP. 

Nous repartons ici sur l'exemple de la factorielle, qui est documentée selon la programmation par contrat (PRE/POST).  



In [24]:
def factorielle(n) :
    """calcule la factorielle de n
    PRE : n est un entier positif
    POST : Renvoie n!
    """
    if n==0 or n==1 :
        return 1
    else :
        return n * factorielle(n-1)

factorielle(3)

6


## Programmation défensive

La programmation défensive est une technique visant à anticiper les mauvaises utilisations éventuelles d'une fonction ou d'une méthode mise à disposition d'autres développeurs.  

En programmation par contrat, si l'utilisateur ne respecte pas la précondition, rien n'est garanti.  Prenons par exemple la fonction factorielle ci-dessus.  Que se passe-t'il si on ne respecte pas la pré-condition et qu'on utilise un entier négatif comme paramètre?  Testez-le par vous-même : c'est une situation qu'il vaut mieux éviter.  Le développeur de la fonction n'est pas en tort, puisque sa précondition est explicite, mais la fonction n'est pas robuste, puisqu'elle peut planter de manière assez radicale en cas de mauvaise utilisation.  

On pourrait donc imaginer prévenir l'utilisateur en cas de mauvaise utilisation, par exemple en utilisant une valeur de retour spéciale.  En C, les fonctions renvoient souvent des codes d'erreur.  On pourrait imaginer la même chose ici, en utilisant par exemple -1 comme code d'erreur.  Voici ce que cela donnerait : 


In [ ]:
def factorielle(n) :
    """calcule la factorielle de n
    PRE : n est un entier
    POST : Renvoie n! si n>=0, sinon, renvoie -1. 
    
    """
    if n < 0 :
        return -1
    if n==0 or n==1 :
        return 1
    else :
        return n * factorielle(n-1)
    
print(factorielle(3))
print(factorielle(-1))

Remarquez que la spécification de la fonction a pu être modifié, et que la pré-condition limitant les valeurs possibles pour le paramètre `n` a été levée : tous les entiers sont acceptés.  

Cette implémentation de la fonction factorielle se veut la plus générique possible tout se voulant robuste. Cette démarche, qui vise à accepter une spécification la plus large possible tout en traitant explicitement les cas d'erreurs s'appelle la **programmation défensive**.  

Attention cependant : Cette implémention, à première vue robuste, souffre encore d'une faiblesse importante.  En effet, l'erreur est signalée à l'utilisateur via une valeur de retour spéciale.  Hélas, les utilisateurs ne vérifient pas toujours les valeurs de retour avant de les utiliser.  Regardons par exemple le cas de figure suivant : 


In [ ]:
x = 3
result = 45 + factorielle(x)
print(result)

Si x est positif, tout se passe comme prévu.  Mais que se passe-t'il dans le cas contraire?  Essayez avec un x négatif : le programme renvoie une valeur, sans que l'utilisateur soit au courant qu'il y a eu un problème!  La réponse renvoyée est fausse et il n'y a pas moyen d'en être averti.  

C'est pour cela que l'utilisation de valeurs de retour spéciales telles que des valeurs négatives n'est pas recommandée.  

L'utilisation de messages d'erreurs n'est pas non plus idéale, car, si ces derniers informent l'utilisateur final de l'occurence d'un problème, le fonctionnement du programme n'est pas spécialement modifié suite à cette erreur. Le développeur utilisateur ne peut pas ajuster son implémentation en fonction de ce cas de figure.  

Pour résoudre ce problème, les langages de programmation modernes possèdent un mécanisme d'erreur propre : Les exceptions.  


## Exceptions

Une exception est un signal qui est lancé par un programme en cours d'exécution lorsqu'un cas particulier (typiquement une erreur) se présente.  

Si rien n'est prévu dans le programme, ce dernier s'arrête de manière un peu brute. Cependant, il est possible d'anticiper la survenue d'une exception au niveau du code, et de gérer le cas particulier de la manière adéquate.  
 
Les mécanismes permettant de lancer et de gérer les exceptions sont expliqués dans les ressources du cours théorique.  

Dans le cadre de notre fonction factorielle, nous allons donc lancer une exception lorsqu'un paramètre négatif est utilisé.  Pour cela, nous devrons préalablement la créer, en étendant la classe Exception. 

Testez ce code en faisant varier la valeur de x : Que se passe-t'il quand il a une valeur négative? 


In [26]:
class NegativeParamException(Exception):
    pass

def factorielle(n) :
    """calcule la factorielle de n
    PRE : n est un entier
    POST : Renvoie n! si n>=0
    RAISES : NegativeParamException si n<0
    
    """
    if n < 0 :
        raise NegativeParamException("Un paramètre négatif a été transmis à la fonction factorielle")
    if n==0 or n==1 :
        return 1
    else :
        return n * factorielle(n-1)

x = -3
print(13 + factorielle(x))

NegativeParamException: Un paramètre négatif a été transmis à la fonction factorielle

On a donc bien un mécanisme d'erreur qui permet d'indiquer précisément d'où vient le problème.  Néanmoins, dans l'exemple ci-dessus, le programme "plante" toujours, et les informations relatives à l'exception sont imprimées de manière un peu brute sur stderr.  

Dans la pratique, en fonction des cas, on préférera parfois poursuivre le programme, en indiquant par exemple dans les logs qu'il y a eu un problème, ou en créant un popup pour avertir l'utilisateur.  Cela doit vraiment être réfléchi au cas par cas : est-ce OK que le programme continue à tourner malgré l'erreur qui s'est produite, ou bien vaut-il mieux l'arrêter pour éviter les incohérences ou les corruptions de données?  

Pour poursuivre sur base de notre exemple, imaginons qu'on souhaite simplement imprimer un message d'erreur propre (sans le "stacktrace") à l'intention de l'utilisateur, sur stdout et non stderr.  Pour cela, on va "interrompre" le plantage du programme en "attrapant" l'exception via les mots-clé "try/except" : 

In [27]:
def test_factorielle(x) : 
    try : 
        print(factorielle(x))
    except NegativeParamException as e: 
        print(e)

test_factorielle(3)
test_factorielle(-1)
test_factorielle(5)

6
Un paramètre négatif a été transmis à la fonction factorielle
120


Notez comme le programme a pu se poursuivre malgré la tentative de calcul d'une factorielle négative.  Notez également que le message d'erreur de l'exception peut être récupéré de manière simple.  

## Mise en pratique 

Vous allez à présent mettre en oeuvre ces trois principes : 
- La programmation par contrat : Spécifier fonctions et méthodes
- La programmation défensive : Avoir un code robuste et des spécifications les plus générales possibles
- Le mécanisme d'exceptions : Utiliser le mécanisme natif de gestion des erreurs pour gérer ces dernières proprement.  


Vous trouverez ci-dessous une classe Fraction.  Cette classe n'est ni spécifiée, ni implémentée.  Il vous est demandé de : 

1. Créer dans pycharm un fichier python pour cette classe, et copiez-y le squelette de code ci-dessous.  

2. Complétez les spécifications de cette classe selon les principes de la programmation par contrat.  Essayez de couvrir un maximum de cas d'utilisation de telle sorte que les spécifications soient les plus larges possibles.  Assurez la robustesse grâce à des exceptions lorsque c'est nécessaire. 

3. Dans un autre fichier python, importez le module fraction, puis écrivez quelques lignes de code créant et utilisant des fractions.  Essayez d'envisager un maximum de cas de figure pour couvrir toutes les méthodes.   Gérez les éventuelles exceptions avec un try/except.  

4. Choisissez-vous un binôme, et échangez vos classes Fraction.  Implémentez chaque méthode de la classe documentée par votre binôme en suivant strictement sa spécification, puis rendez cette classe à son créateur. (Remarque : L'élement le plus difficile dans cette implémentation est la simplification de fraction (reduce en anglais).  Vous trouverez facilement sur Google l'algorithme d'Euclide pour le calcul du PGCD). 

5.  Testez l'implémentation de votre classe par votre binôme à l'aide de votre script du point 3. 


### Recommandations

- N'utilisez pas le module "fractions" de Python, cela ôterait tout intérêt à l'exercice (et votre script sera refusé pour évaluation)
- Simplifiez les fractions dès que possible.  Les calculs sont plus simples si vous faites en sorte que la représentation interne de la fraction est toujours simplifiée, dès sa création. 
- Votre implémentation doit effectuer des calculs "exacts", sans passer par des conversions vers les valeurs numériques (ex : Faire 1/4 + 1/3 en convertissant en float 0,25 + 0,33).  La mise au même dénominateur vous sera indispensable
- Les opérations s'effectuent par défaut entre des objets Fraction (donc entre l'objet "self" et un objet "other" passé en paramètres.  Si vous souhaitez implémenter des opérations qui couvrent d'autres types de données (par ex. l'addition entre une fraction et un entier), cela doit être en conformité avec la spécification de la méthode.
- Vous pouvez utiliser les "annotations" Python pour indiquer le type des paramètres et des valeurs de retour.  Cela permet de simplifier les spécifications. 
- N'oubliez pas qu'une des quatre opérations arithmétiques peut poser problème avec une valeur particulière! 
- Exploitez au maximum le mécanisme des exceptions. 


In [ ]:
class Fraction:
    """Class representing a fraction and operations on it

    Author : V. Van den Schrieck
    Date : October 2021
    This class allows fraction manipulations through several operations.
    """

    def __init__(num=0, den=1):
        """This builds a fraction based on some numerator and denominator.

        PRE : ?
        POST : ?
        """
        pass

    @property
    def numerator(self):
        pass
    @property
    def denominator(self):
        pass

# ------------------ Textual representations ------------------

    def __str__(self) :
        """Return a textual representation of the reduced form of the fraction

        PRE : ?
        POST : ?
        """
        pass

    def as_mixed_number(self) :
        """Return a textual representation of the reduced form of the fraction as a mixed number

        A mixed number is the sum of an integer and a proper fraction

        PRE : ?
        POST : ?
        """
        pass

    
# ------------------ Operators overloading ------------------

    def __add__(self, other):
        """Overloading of the + operator for fractions

         PRE : ?
         POST : ?
         """
        pass


    def __sub__(self, other):
        """Overloading of the - operator for fractions

        PRE : ?
        POST : ?
        """
        pass


    def __mul__(self, other):
        """Overloading of the * operator for fractions

        PRE : ?
        POST : ?
        """
        pass


    def __truediv__(self, other):
        """Overloading of the / operator for fractions

        PRE : ?
        POST : ?
        """
        pass


    def __pow__(self, other):
        """Overloading of the ** operator for fractions

        PRE : ?
        POST : ?
        """
        pass
    
    
    def __eq__(self, other) : 
        """Overloading of the == operator for fractions
        
        PRE : ?
        POST : ? 
        
        """
        
    def __float__(self) :
        """Returns the decimal value of the fraction

        PRE : ?
        POST : ?
        """
        pass
    
# TODO : [BONUS] You can overload other operators if you wish (ex : <, >, ...)




# ------------------ Properties checking  ------------------

    def is_zero(self):
        """Check if a fraction's value is 0

        PRE : ?
        POST : ?
        """
        pass


    def is_integer(self):
        """Check if a fraction is integer (ex : 8/4, 3, 2/2, ...)

        PRE : ?
        POST : ?
        """
        pass

    def is_proper(self):
        """Check if the absolute value of the fraction is < 1

        PRE : ?
        POST : ?
        """

    def is_unit(self):
        """Check if a fraction's numerator is 1 in its reduced form

        PRE : ?
        POST : ?
        """
        pass

    def is_adjacent_to(self, other) :
        """Check if two fractions differ by a unit fraction

        Two fractions are adjacents if the absolute value of the difference is a unit fraction

        PRE : ?
        POST : ?
        """
        pass

